# Kaggle : prédiction sur des données de restaurants

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Se positionner dans le repertoire courant

In [ ]:
cd /content/drive/My Drive/Kaggle/esieedsia2020

/content/drive/My Drive/Kaggle/esieedsia2020


Import de toutes les librairies nécessaires

In [ ]:
# Imports des librairies
import datetime
import sklearn
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyoff
import xgboost as xgb
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import date
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_predict
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB, GaussianNB

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Chargement des fichier csv en dataframe

In [ ]:
# Imports des csv

df_train = pd.read_csv("Train.csv")
df_test=pd.read_csv("Train.csv")
df_members = pd.read_csv("member.csv",delimiter='\t',encoding="UTF-16")
df_restaurants = pd.read_csv("restaurant.csv",delimiter='\t',encoding="UTF-16")
df_submission=pd.read_csv("submission_example.csv")     

# Nettoyage de la table **Train**

**Pour commencer, intéressons-nous aux données.**
- Corrélation des variables prédictives avec la variable à prédire.
- Affichage de la répartition des données pour plusieurs variables (ex: gender, datetime, purpose, return90)
- Une fonction regroupant tout le nettoyage sera faites dans la suite.

In [ ]:
df_train.head()
#df_train.info()

,booking_id,member_id,cdate,restaurant_id,datetime,people,purpose,gender,status,is_required_prepay_satisfied,return90
0,154664,ec9352fdbcb8df88fbf98f964431c063d226e96e,"1/17/2014,,2:54:00,AM",dea8ce93bfe3b40da0cca953fd9d30811d680373,"1/23/2014,,3:00:00,PM",2,Sweet day,F,ok,1,1
1,99188,97fa82b7632705e970530cacebfe841e908d04b6,"4/28/2012,,11:10:00,PM",f3837c4489cc6c4587dfb8a92a060585c182ccc1,"4/30/2012,,11:30:00,AM",2,Dîner entre amis,M,ok,1,0
2,142140,d9643f4373b465fd8ca5fe75fffdbfd161f7a8df,"2/1/2013,,5:25:00,PM",c460d6b5860c01ad97d3ef5d9886548361e5c78e,"2/4/2013,,7:00:00,PM",2,Dîner entre amis,M,ok,1,0
3,128744,c4baf4ebccdf1a7983ca34673f4b06d995a6eae9,"8/25/2012,,1:53:00,PM",52f8e137046607186800ca3c01a954e928c16412,"8/25/2012,,6:30:00,PM",9,Repas en famille,F,ok,1,0
4,134126,cd16b9904eeee3585eb83d1d19e8ffbeee2b0327,"11/24/2013,,10:47:00,AM",fe230aa38ebf31c978d0f58bd6fd5ab9e7ec1e66,"12/8/2013,,3:00:00,PM",4,Repas en famille,F,canceled,1,1


In [ ]:
df_train.dtypes

booking_id                       int64
member_id                       object
cdate                           object
restaurant_id                   object
datetime                        object
people                           int64
purpose                         object
gender                          object
status                          object
is_required_prepay_satisfied     int64
return90                         int64
dtype: object

In [ ]:
# missing values
total = df_train.isnull().sum().sort_values(ascending=False)
# show
pd.DataFrame(data={'Missing': total})

,Missing
purpose,5529
return90,0
is_required_prepay_satisfied,0
status,0
gender,0
people,0
datetime,0
restaurant_id,0
cdate,0
member_id,0


On remplace les Nan values dans la colonne Purpose par Non Renseigné.

In [ ]:
df_train['purpose'].replace(np.nan,'Non renseigné',inplace=True)

In [ ]:
# Type des variables
df_train.dtypes

booking_id                       int64
member_id                       object
cdate                           object
restaurant_id                   object
datetime                        object
people                           int64
purpose                         object
gender                          object
status                          object
is_required_prepay_satisfied     int64
return90                         int64
dtype: object

In [ ]:
# Histogramme de la répartition de la variable à prédire
plot_data = go.Bar(
            x = df_train["return90"].value_counts().tolist(),
            y= df_train["return90"].value_counts().index.tolist(),
            orientation='h')

plot_layout = go.Layout(
        title='Répartition de la variable à prédire "return90"'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)

pyoff.iplot(fig)

In [ ]:
# Histogramme des membres en fonction de leur genre
plot_data = go.Bar(
            x = df_train["gender"].value_counts().tolist(),
            y= df_train["gender"].value_counts().index.tolist(),
            orientation='h')

plot_layout = go.Layout(
        title='Member by gender'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)

pyoff.iplot(fig)

In [ ]:
# Histogramme des réservations en fonction du purpose
plot_data = go.Bar(
            x = df_train["purpose"].value_counts().tolist(),
            y= df_train["purpose"].value_counts().index.tolist(),
            orientation='h')

plot_layout = go.Layout(
        title='Number of reservation by purpose'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)

pyoff.iplot(fig)

In [ ]:
df_train.purpose.value_counts(dropna=False)

Dîner entre amis                                    27448
Repas en famille                                    20422
Fête d'anniversaire                                 16470
Sweet day                                            5936
Non renseigné                                        5529
Dîner d'affaires                                     1669
Date importante                                       313
Veuillez sélectionner                                 100
Célébration d'anniversaire                             53
其他                                                     23
Friends,Reunion                                         8
Please,Select                                           4
Family,Gathering                                        1
Birthday                                                1
&#65533;&#40115;&#65533;&#65533;&#26813;&#65533;        1
Repas en famille,盡量靠W                                   1
Repas en famille-生日                                     1
家人&#65533;    

On traite la colonne date. On va séparer la date et l'heure à laquelle les personnes sont venues manger car il est possible que l'heure (midi ou soir) et la saison peuvent etre determinants dans la prédiction.

In [ ]:
df_train['datetime']=pd.to_datetime(df_train['datetime'])
df_train['lunch_year']=df_train['datetime'].map(lambda x: x.strftime('%Y')).astype(int)
df_train['lunch_month']=df_train['datetime'].map(lambda x: x.strftime('%m')).astype(int)
df_train['lunch_day']=df_train['datetime'].map(lambda x: x.strftime('%d')).astype(int)
df_train['lunch_hour']=df_train['datetime'].map(lambda x: x.strftime('%H')).astype(int)
df_train['lunch_minute']=df_train['datetime'].map(lambda x: x.strftime('%M')).astype(int)

df_train=df_train.drop(labels=['datetime'],axis=1)
df_train=df_train.rename(columns={'cdate':'reservation_date'})
df_train['reservation_date']=pd.to_datetime(df_train['reservation_date'])
df_train['reservation_date_year']=df_train['reservation_date'].map(lambda x: x.strftime('%Y')).astype(int)
df_train['reservation_date_month']=df_train['reservation_date'].map(lambda x: x.strftime('%m')).astype(int)
df_train['reservation_date_day']=df_train['reservation_date'].map(lambda x: x.strftime('%d')).astype(int)
df_train['reservation_date_hour']=df_train['reservation_date'].map(lambda x: x.strftime('%H')).astype(int)
df_train['reservation_date_minute']=df_train['reservation_date'].map(lambda x: x.strftime('%M')).astype(int)
df_train=df_train.drop(labels=['reservation_date'],axis=1)
df_train.head()

,booking_id,member_id,restaurant_id,people,purpose,gender,status,is_required_prepay_satisfied,return90,lunch_year,lunch_month,lunch_day,lunch_hour,lunch_minute,reservation_date_year,reservation_date_month,reservation_date_day,reservation_date_hour,reservation_date_minute
0,154664,ec9352fdbcb8df88fbf98f964431c063d226e96e,dea8ce93bfe3b40da0cca953fd9d30811d680373,2,Sweet day,F,ok,1,1,2014,1,23,15,0,2014,1,17,2,54
1,99188,97fa82b7632705e970530cacebfe841e908d04b6,f3837c4489cc6c4587dfb8a92a060585c182ccc1,2,Dîner entre amis,M,ok,1,0,2012,4,30,11,30,2012,4,28,23,10
2,142140,d9643f4373b465fd8ca5fe75fffdbfd161f7a8df,c460d6b5860c01ad97d3ef5d9886548361e5c78e,2,Dîner entre amis,M,ok,1,0,2013,2,4,19,0,2013,2,1,17,25
3,128744,c4baf4ebccdf1a7983ca34673f4b06d995a6eae9,52f8e137046607186800ca3c01a954e928c16412,9,Repas en famille,F,ok,1,0,2012,8,25,18,30,2012,8,25,13,53
4,134126,cd16b9904eeee3585eb83d1d19e8ffbeee2b0327,fe230aa38ebf31c978d0f58bd6fd5ab9e7ec1e66,4,Repas en famille,F,canceled,1,1,2013,12,8,15,0,2013,11,24,10,47


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77982 entries, 0 to 77981
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   booking_id                    77982 non-null  int64 
 1   member_id                     77982 non-null  object
 2   restaurant_id                 77982 non-null  object
 3   people                        77982 non-null  int64 
 4   purpose                       77982 non-null  object
 5   gender                        77982 non-null  object
 6   status                        77982 non-null  object
 7   is_required_prepay_satisfied  77982 non-null  int64 
 8   return90                      77982 non-null  int64 
 9   lunch_year                    77982 non-null  int64 
 10  lunch_month                   77982 non-null  int64 
 11  lunch_day                     77982 non-null  int64 
 12  lunch_hour                    77982 non-null  int64 
 13  lunch_minute    

On transforme la colonne gender en colonne binaire.

In [ ]:
Gender = {'M': 1,'F': 0}
df_train.gender = [Gender[item] for item in df_train.gender]
df_train.head()

,booking_id,member_id,restaurant_id,people,purpose,gender,status,is_required_prepay_satisfied,return90,lunch_year,lunch_month,lunch_day,lunch_hour,lunch_minute,reservation_date_year,reservation_date_month,reservation_date_day,reservation_date_hour,reservation_date_minute
0,154664,ec9352fdbcb8df88fbf98f964431c063d226e96e,dea8ce93bfe3b40da0cca953fd9d30811d680373,2,Sweet day,0,ok,1,1,2014,1,23,15,0,2014,1,17,2,54
1,99188,97fa82b7632705e970530cacebfe841e908d04b6,f3837c4489cc6c4587dfb8a92a060585c182ccc1,2,Dîner entre amis,1,ok,1,0,2012,4,30,11,30,2012,4,28,23,10
2,142140,d9643f4373b465fd8ca5fe75fffdbfd161f7a8df,c460d6b5860c01ad97d3ef5d9886548361e5c78e,2,Dîner entre amis,1,ok,1,0,2013,2,4,19,0,2013,2,1,17,25
3,128744,c4baf4ebccdf1a7983ca34673f4b06d995a6eae9,52f8e137046607186800ca3c01a954e928c16412,9,Repas en famille,0,ok,1,0,2012,8,25,18,30,2012,8,25,13,53
4,134126,cd16b9904eeee3585eb83d1d19e8ffbeee2b0327,fe230aa38ebf31c978d0f58bd6fd5ab9e7ec1e66,4,Repas en famille,0,canceled,1,1,2013,12,8,15,0,2013,11,24,10,47


On regroupe certaines valeurs de la colonne **purpose** qui appartiennent au même raison de réservation.

In [ ]:
# Gather some values
df_train = df_train.replace(dict.fromkeys(['Repas en famille','Repas en famille,盡量靠W','Repas en famille-生日','Family,Gathering',"Repas en f"], 'Famille'))
df_train = df_train.replace(dict.fromkeys(['Dîner entre amis','Friends,Reunion'], 'Amis'))
df_train = df_train.replace(dict.fromkeys(["Fête d'anniversaire",'Birthday,Celebration','Birthday',"Célébration d'anniversaire"], 'Anniversaire'))
df_train = df_train.replace(dict.fromkeys(['Sweet day','Date importante'], 'Date'))
df_train = df_train.replace(dict.fromkeys(["Dîner d'affaires"], 'Affaire'))
# For values with less than 80 occurences replace by 'Autres'
df_train = df_train.replace(dict.fromkeys(['Veuillez sélectionner','其他', 'Please,Select',
                                    '&#65533;&#40115;&#65533;&#65533;&#26813;&#65533;', '家人&#65533;'], 'Autre'))


In [ ]:
df_train.purpose.value_counts(dropna=False)

Amis             27456
Famille          20425
Anniversaire     16525
Date              6249
Non renseigné     5529
Affaire           1669
Autre              129
Name: purpose, dtype: int64

On nous dit dans la description des données que 'new' et 'ok' pour le **statut** appartiennent à la même catégorie.

In [ ]:
df_train = df_train.replace(dict.fromkeys(['new','ok'], 'ok'))
df_train.status.value_counts(dropna=False)

ok          60365
canceled    15337
no-show      1144
changed      1136
Name: status, dtype: int64

# Nettoyage table **members**

In [ ]:
df_members.shape

(699200, 9)

In [ ]:
df_members.dtypes

id               object
is_vip            int64
gender           object
birthdate        object
city             object
has_google_id     int64
has_yahoo_id      int64
has_weibo_id      int64
cdate            object
dtype: object

In [ ]:
df_members.head()

,id,is_vip,gender,birthdate,city,has_google_id,has_yahoo_id,has_weibo_id,cdate
0,e732bcf914fecc5a20adc8940689e701160772c4,0,F,5/14/1981,台北市,0,0,0,4/1/2012 0:01
1,c671d1bd0b03d682d590275f929b613426dc0e34,0,M,3/22/1975,台北市,0,0,0,1/1/2012 1:22
2,fe65711316d561f420bfa3364edf60f0c1859d1f,0,F,11/1/1976,0,0,0,0,1/1/2012 1:32
3,6113c28cfc2cf86ae268657683eca4953df54342,0,M,0000-00-00,0,0,0,0,1/1/2012 1:33
4,2fc08e00a891d491c99323aed57c4cb53922c959,0,F,1/1/1977,0,0,0,0,1/1/2012 1:54


In [ ]:
# missing values
total_members = df_members.isnull().sum().sort_values(ascending=False)
# show
pd.DataFrame(data={'Missing': total_members})

,Missing
city,478
gender,42
cdate,0
has_weibo_id,0
has_yahoo_id,0
has_google_id,0
birthdate,0
is_vip,0
id,0


On extrait de la date de naissance l'**âge** du membre et on renomme la colonne. 0 sera la valeur pour l'âge non renseigné.

In [ ]:
df_members['birthdate']=df_members["birthdate"].str.replace("-","/")
df_members['birthdate']=df_members["birthdate"].str.split("/").str[2]
df_members['birthdate']=df_members['birthdate'].astype(int)
df_members['birthdate']=2020-df_members['birthdate']
df_members=df_members.rename(columns={'birthdate':'age'})
df_members['age']=df_members['age'].replace(2020,0)
df_members.head()

,id,is_vip,gender,age,city,has_google_id,has_yahoo_id,has_weibo_id,cdate
0,e732bcf914fecc5a20adc8940689e701160772c4,0,F,39,台北市,0,0,0,4/1/2012 0:01
1,c671d1bd0b03d682d590275f929b613426dc0e34,0,M,45,台北市,0,0,0,1/1/2012 1:22
2,fe65711316d561f420bfa3364edf60f0c1859d1f,0,F,44,0,0,0,0,1/1/2012 1:32
3,6113c28cfc2cf86ae268657683eca4953df54342,0,M,0,0,0,0,0,1/1/2012 1:33
4,2fc08e00a891d491c99323aed57c4cb53922c959,0,F,43,0,0,0,0,1/1/2012 1:54


On s'occupe ici du champ **cdate** qu'on convertit en datetime et on supprime l'heure. On l'appelle **mbr_record_date** pour que ce soit plus intuitif. De plus on extrait l'anciennté du membre. On change aussi **id** en **member_id** pour faire la jointure avec la table **Train**.

In [ ]:
df_members['cdate']=pd.to_datetime(df_members['cdate'])
df_members['mbr_record_date_year']=df_members['cdate'].map(lambda x: x.strftime('%Y')).astype(int)
df_members['mbr_record_date_month']=df_members['cdate'].map(lambda x: x.strftime('%m')).astype(int)
df_members['mbr_record_date_day']=df_members['cdate'].map(lambda x: x.strftime('%d')).astype(int)
df_members=df_members.drop(labels=['cdate'],axis=1)
df_members=df_members.rename(columns={'id':'member_id'})
df_members.head()

,member_id,is_vip,gender,age,city,has_google_id,has_yahoo_id,has_weibo_id,mbr_record_date_year,mbr_record_date_month,mbr_record_date_day
0,e732bcf914fecc5a20adc8940689e701160772c4,0,F,39,台北市,0,0,0,2012,4,1
1,c671d1bd0b03d682d590275f929b613426dc0e34,0,M,45,台北市,0,0,0,2012,1,1
2,fe65711316d561f420bfa3364edf60f0c1859d1f,0,F,44,0,0,0,0,2012,1,1
3,6113c28cfc2cf86ae268657683eca4953df54342,0,M,0,0,0,0,0,2012,1,1
4,2fc08e00a891d491c99323aed57c4cb53922c959,0,F,43,0,0,0,0,2012,1,1


Pour la colonne Gender on peut remplacer les NaN par la valeur la plus fréquente qui est F. De plus on peut directement remplacer F et M par 0 et 1 pour avoir une seule colonne binaire. 

In [ ]:
df_members['gender']=df_members['gender'].fillna('F')
Gender = {'M': 1,'F': 0}
df_members.gender = [Gender[item] for item in df_members.gender]

df_members.head()

,member_id,is_vip,gender,age,city,has_google_id,has_yahoo_id,has_weibo_id,mbr_record_date_year,mbr_record_date_month,mbr_record_date_day
0,e732bcf914fecc5a20adc8940689e701160772c4,0,0,39,台北市,0,0,0,2012,4,1
1,c671d1bd0b03d682d590275f929b613426dc0e34,0,1,45,台北市,0,0,0,2012,1,1
2,fe65711316d561f420bfa3364edf60f0c1859d1f,0,0,44,0,0,0,0,2012,1,1
3,6113c28cfc2cf86ae268657683eca4953df54342,0,1,0,0,0,0,0,2012,1,1
4,2fc08e00a891d491c99323aed57c4cb53922c959,0,0,43,0,0,0,0,2012,1,1


On supprime les colonnes inutiles.

In [ ]:
df_members=df_members.drop(['has_yahoo_id','has_weibo_id','has_google_id'],axis=1)

Pour la colonne **city**, on voit que lorsque la ville n'est pas renseignée cela prend la valeur 0 donc on remplace nos Nan par 0. De plus on décide de remplacer le nom de la colonne par **residence** pour que ce soit plus intuitif.

In [ ]:
df_members['city']=df_members['city'].fillna('0')
df_members=df_members.rename(columns={'city':'residence'})

# Nettoyage de la table **restaurant**

In [ ]:
df_restaurants.head()

,id,is_hotel,country,currency,city,cityarea,name,abbr,tel,opening_hours,good_for_family,accept_credit_card,parking,outdoor_seating,wifi,wheelchair_accessible,price1,price2,lat,lng,timezone,locale,cdate
0,fb1171550a7ad65f5533d24345e7677c5353168d,1,tw,TWD,新北市,NaN,db231778a1d2a260b510e4379dfb159a63e90e95,e0199e43c23c7322f91049f1c49a9c8d87b3bc39,50f5512c6ed9d5e1b8112e7e52fa3d2aa76514ea,24小時營業(湯屋每次為雙人使用，加時與加人依現場報價為準),1,1,1,0,1,1,1100,2200,25.20,121.60,Asia/Taipei,zh_TW,2/27/2013 18:25
1,8239c2381e09ba5a298eb9ebefa88de9976105a5,0,tw,TWD,新北市,NaN,da4531bc68b42dff77fffc7f1b748bfd2a1855b7,da4531bc68b42dff77fffc7f1b748bfd2a1855b7,44d48f31e6b7aa2184018b7d0156fd8e068e3368,週二至週日\r\n午餐：11:00~14:00 \r\n晚餐：17:00~ 21:00\r\...,1,1,1,0,1,0,300,400,25.00,121.52,Asia/Taipei,zh_TW,8/26/2014 13:37
2,85709eabedeb5c761b424baa16db3bccd6144fd4,1,tw,TWD,台中市,請選擇,e1bb633022c44a15d15a072e17274887707e5774,e1bb633022c44a15d15a072e17274887707e5774,a43a6b1ab17d261040e844ad4db619924c58d188,營業時間：06:30~23:30\r\n\r\n,1,1,1,0,1,1,563,736,24.23,120.94,Asia/Taipei,zh_TW,2/25/2012 20:47
3,8a035d8ccfaae36efff1cdcc6f02f318f35abd64,1,tw,TWD,台中市,烏日區,2fb3f01285d5d64069513e2f2a06e322e352dcdc,2fb3f01285d5d64069513e2f2a06e322e352dcdc,353015f4db1660c7060c3e9a3063097aff22efa2,平日早餐：6:30- 10:00; \r\n午餐：11:30- 14:00（14:00清場)...,1,1,1,0,1,1,379,564,24.14,120.59,Asia/Taipei,zh_TW,6/25/2012 11:59
4,56a3a7c9675c5aca5e98ea3d9aa77610ec04183a,0,tw,TWD,台中市,NaN,408bce81d566a17affd9b56344e0dfd111c69548,408bce81d566a17affd9b56344e0dfd111c69548,caeaa8587bfb7b04cc0d6c1fcf15099decdba1cd,午餐:11:30~14:00\r\n午茶:14:30~16:30\r\n晚餐:17:30~2...,1,1,1,0,1,0,499,1000,24.23,120.94,Asia/Taipei,zh_TW,8/15/2013 16:56


In [ ]:
df_restaurants.dtypes

id                        object
is_hotel                   int64
country                   object
currency                  object
city                      object
cityarea                  object
name                      object
abbr                      object
tel                       object
opening_hours             object
good_for_family            int64
accept_credit_card         int64
parking                    int64
outdoor_seating            int64
wifi                       int64
wheelchair_accessible      int64
price1                     int64
price2                     int64
lat                      float64
lng                      float64
timezone                  object
locale                    object
cdate                     object
dtype: object

In [ ]:
# missing values
total_restaurants = df_restaurants.isnull().sum().sort_values(ascending=False)
# show
pd.DataFrame(data={'Missing': total_restaurants})

,Missing
cityarea,362
cdate,0
good_for_family,0
is_hotel,0
country,0
currency,0
city,0
name,0
abbr,0
tel,0


On voit que pour cette variable il y a plus de Nan que de valeur renseignée donc il faudra vérifier si cette valeur est important ou non pour la supprimer apres.
Pour le moment on remplace pr la valeur la plus vue.

In [ ]:
df_restaurants['cityarea']=df_restaurants['cityarea'].fillna("請選擇")
#df_restaurants.cityarea.value_counts(dropna=False)

On modifie la colonne cdate. On la renomme add_restau_date pour que cela soit plus intuitif et on extrait l'ancienneté du restaurant. on renomme la colonne id en **restaurant_id** pour pouvoir faire la jointure avec la table train.

In [ ]:
df_restaurants['cdate']=pd.to_datetime(df_restaurants['cdate'])
df_restaurants['restau_add_date_year']=df_restaurants['cdate'].map(lambda x: x.strftime('%Y')).astype(int)
df_restaurants['restau_add_date_month']=df_restaurants['cdate'].map(lambda x: x.strftime('%m')).astype(int)
df_restaurants['restau_add_date_day']=df_restaurants['cdate'].map(lambda x: x.strftime('%d')).astype(int)
df_restaurants=df_restaurants.drop(labels=['cdate'],axis=1)
df_restaurants=df_restaurants.rename(columns={'id':'restaurant_id'})
df_restaurants.head()
#df_restaurants.info()

,restaurant_id,is_hotel,country,currency,city,cityarea,name,abbr,tel,opening_hours,good_for_family,accept_credit_card,parking,outdoor_seating,wifi,wheelchair_accessible,price1,price2,lat,lng,timezone,locale,restau_add_date_year,restau_add_date_month,restau_add_date_day
0,fb1171550a7ad65f5533d24345e7677c5353168d,1,tw,TWD,新北市,請選擇,db231778a1d2a260b510e4379dfb159a63e90e95,e0199e43c23c7322f91049f1c49a9c8d87b3bc39,50f5512c6ed9d5e1b8112e7e52fa3d2aa76514ea,24小時營業(湯屋每次為雙人使用，加時與加人依現場報價為準),1,1,1,0,1,1,1100,2200,25.20,121.60,Asia/Taipei,zh_TW,2013,2,27
1,8239c2381e09ba5a298eb9ebefa88de9976105a5,0,tw,TWD,新北市,請選擇,da4531bc68b42dff77fffc7f1b748bfd2a1855b7,da4531bc68b42dff77fffc7f1b748bfd2a1855b7,44d48f31e6b7aa2184018b7d0156fd8e068e3368,週二至週日\r\n午餐：11:00~14:00 \r\n晚餐：17:00~ 21:00\r\...,1,1,1,0,1,0,300,400,25.00,121.52,Asia/Taipei,zh_TW,2014,8,26
2,85709eabedeb5c761b424baa16db3bccd6144fd4,1,tw,TWD,台中市,請選擇,e1bb633022c44a15d15a072e17274887707e5774,e1bb633022c44a15d15a072e17274887707e5774,a43a6b1ab17d261040e844ad4db619924c58d188,營業時間：06:30~23:30\r\n\r\n,1,1,1,0,1,1,563,736,24.23,120.94,Asia/Taipei,zh_TW,2012,2,25
3,8a035d8ccfaae36efff1cdcc6f02f318f35abd64,1,tw,TWD,台中市,烏日區,2fb3f01285d5d64069513e2f2a06e322e352dcdc,2fb3f01285d5d64069513e2f2a06e322e352dcdc,353015f4db1660c7060c3e9a3063097aff22efa2,平日早餐：6:30- 10:00; \r\n午餐：11:30- 14:00（14:00清場)...,1,1,1,0,1,1,379,564,24.14,120.59,Asia/Taipei,zh_TW,2012,6,25
4,56a3a7c9675c5aca5e98ea3d9aa77610ec04183a,0,tw,TWD,台中市,請選擇,408bce81d566a17affd9b56344e0dfd111c69548,408bce81d566a17affd9b56344e0dfd111c69548,caeaa8587bfb7b04cc0d6c1fcf15099decdba1cd,午餐:11:30~14:00\r\n午茶:14:30~16:30\r\n晚餐:17:30~2...,1,1,1,0,1,0,499,1000,24.23,120.94,Asia/Taipei,zh_TW,2013,8,15


On supprime les colonnes inutiles.

In [ ]:
df_restaurants=df_restaurants.drop(["timezone","locale","lat","lng"],axis=1)

# Jointures entre les tables

On joint la table member et restaurant à la table Train sur les clés **member_id** et **restaurant_id** tout en gardant les autres colonnes qui ne matchent pas. Ensuite on garde que les lignes avec un **booking_id**. 

In [ ]:
df_train_restaurant=pd.merge(df_train,df_restaurants,on=['restaurant_id'],how='outer')
df_train_merge=pd.merge(df_train_restaurant,df_members,on=['member_id'],how='outer')
df_train_final=df_train_merge[df_train_merge['booking_id'].notnull()]
df_train_final.head()

,booking_id,member_id,restaurant_id,people,purpose,gender_x,status,is_required_prepay_satisfied,return90,lunch_year,lunch_month,lunch_day,lunch_hour,lunch_minute,reservation_date_year,reservation_date_month,reservation_date_day,reservation_date_hour,reservation_date_minute,is_hotel,country,currency,city,cityarea,name,abbr,tel,opening_hours,good_for_family,accept_credit_card,parking,outdoor_seating,wifi,wheelchair_accessible,price1,price2,restau_add_date_year,restau_add_date_month,restau_add_date_day,is_vip,gender_y,age,residence,mbr_record_date_year,mbr_record_date_month,mbr_record_date_day
0,154664.0,ec9352fdbcb8df88fbf98f964431c063d226e96e,dea8ce93bfe3b40da0cca953fd9d30811d680373,2.0,Date,0.0,ok,1.0,1.0,2014.0,1.0,23.0,15.0,0.0,2014.0,1.0,17.0,2.0,54.0,1.0,tw,TWD,台北市,請選擇,9fb8c9229bd023c0583bdc068fe4a57dedb25d30,36a4d0c7584cf8e97b517f4290295346529cdd7b,841286c917160869956579588d070e19ba2072ac,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,0.0,32.0,0,2014.0,1.0,9.0
1,94757.0,91183899754c45533e3acf90b891346be7dfe2af,dea8ce93bfe3b40da0cca953fd9d30811d680373,2.0,Non renseigné,0.0,ok,1.0,0.0,2014.0,2.0,9.0,18.0,30.0,2014.0,2.0,5.0,14.0,12.0,1.0,tw,TWD,台北市,請選擇,9fb8c9229bd023c0583bdc068fe4a57dedb25d30,36a4d0c7584cf8e97b517f4290295346529cdd7b,841286c917160869956579588d070e19ba2072ac,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,0.0,35.0,0,2014.0,2.0,5.0
2,122395.0,bb50f0f43deaf0426c3d5c97d2c1c587cfdc9846,dea8ce93bfe3b40da0cca953fd9d30811d680373,2.0,Date,1.0,canceled,1.0,1.0,2014.0,6.0,21.0,11.0,30.0,2014.0,6.0,17.0,22.0,50.0,1.0,tw,TWD,台北市,請選擇,9fb8c9229bd023c0583bdc068fe4a57dedb25d30,36a4d0c7584cf8e97b517f4290295346529cdd7b,841286c917160869956579588d070e19ba2072ac,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,1.0,0.0,0,2014.0,5.0,20.0
3,104423.0,a0013dd17729acc6f5bd919196c3301a2d7e311d,dea8ce93bfe3b40da0cca953fd9d30811d680373,2.0,Anniversaire,0.0,ok,1.0,0.0,2013.0,9.0,21.0,15.0,0.0,2013.0,9.0,21.0,8.0,41.0,1.0,tw,TWD,台北市,請選擇,9fb8c9229bd023c0583bdc068fe4a57dedb25d30,36a4d0c7584cf8e97b517f4290295346529cdd7b,841286c917160869956579588d070e19ba2072ac,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,1.0,7.0,台北市,2012.0,8.0,15.0
4,91986.0,8ca56c95f5c737b6d2af31023408db7859a71e48,dea8ce93bfe3b40da0cca953fd9d30811d680373,4.0,Famille,0.0,ok,1.0,0.0,2014.0,2.0,26.0,11.0,30.0,2014.0,2.0,19.0,21.0,47.0,1.0,tw,TWD,台北市,請選擇,9fb8c9229bd023c0583bdc068fe4a57dedb25d30,36a4d0c7584cf8e97b517f4290295346529cdd7b,841286c917160869956579588d070e19ba2072ac,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On peut maintenant enlever tous les identifiants et les informations sur le restaurant tel que le nom, le numero de telephone et l'adresse.

In [ ]:
df_train_final=df_train_final.drop(labels=['member_id','restaurant_id','name','tel','abbr','gender_y','country'],axis=1)
df_train_final=df_train_final.rename(columns={'gender_x':'gender'})
df_train_final.head()

,booking_id,people,purpose,gender,status,is_required_prepay_satisfied,return90,lunch_year,lunch_month,lunch_day,lunch_hour,lunch_minute,reservation_date_year,reservation_date_month,reservation_date_day,reservation_date_hour,reservation_date_minute,is_hotel,currency,city,cityarea,opening_hours,good_for_family,accept_credit_card,parking,outdoor_seating,wifi,wheelchair_accessible,price1,price2,restau_add_date_year,restau_add_date_month,restau_add_date_day,is_vip,age,residence,mbr_record_date_year,mbr_record_date_month,mbr_record_date_day
0,154664.0,2.0,Date,0.0,ok,1.0,1.0,2014.0,1.0,23.0,15.0,0.0,2014.0,1.0,17.0,2.0,54.0,1.0,TWD,台北市,請選擇,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,32.0,0,2014.0,1.0,9.0
1,94757.0,2.0,Non renseigné,0.0,ok,1.0,0.0,2014.0,2.0,9.0,18.0,30.0,2014.0,2.0,5.0,14.0,12.0,1.0,TWD,台北市,請選擇,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,35.0,0,2014.0,2.0,5.0
2,122395.0,2.0,Date,1.0,canceled,1.0,1.0,2014.0,6.0,21.0,11.0,30.0,2014.0,6.0,17.0,22.0,50.0,1.0,TWD,台北市,請選擇,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,0.0,0,2014.0,5.0,20.0
3,104423.0,2.0,Anniversaire,0.0,ok,1.0,0.0,2013.0,9.0,21.0,15.0,0.0,2013.0,9.0,21.0,8.0,41.0,1.0,TWD,台北市,請選擇,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,0.0,7.0,台北市,2012.0,8.0,15.0
4,91986.0,4.0,Famille,0.0,ok,1.0,0.0,2014.0,2.0,26.0,11.0,30.0,2014.0,2.0,19.0,21.0,47.0,1.0,TWD,台北市,請選擇,早餐 自助餐 06:30 am - 10:30 am\r\n午餐 自助餐 11:...,1.0,1.0,1.0,0.0,1.0,1.0,1100.0,1500.0,2013.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN


# Remplacement des valeurs Nan
 Etant donné qu'il existe encore des valeurs nulles dans certaines colonnes, on va remplacer ces valeurs nulles par des valeurs stratégiques.

In [ ]:
df_train_final['residence']=df_train_final['residence'].fillna('0')
df_train_final['price1']=df_train_final['price1'].fillna(df_train_final['price1'].mean())
df_train_final['price2']=df_train_final['price2'].fillna(df_train_final['price2'].mean())
df_train_final['age']=df_train_final['age'].fillna(df_train_final['age'].mean())
df_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77982 entries, 0 to 77981
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   booking_id                    77982 non-null  float64
 1   people                        77982 non-null  float64
 2   purpose                       77982 non-null  object 
 3   gender                        77982 non-null  float64
 4   status                        77982 non-null  object 
 5   is_required_prepay_satisfied  77982 non-null  float64
 6   return90                      77982 non-null  float64
 7   lunch_year                    77982 non-null  float64
 8   lunch_month                   77982 non-null  float64
 9   lunch_day                     77982 non-null  float64
 10  lunch_hour                    77982 non-null  float64
 11  lunch_minute                  77982 non-null  float64
 12  reservation_date_year         77982 non-null  float64
 13  r

On remplace les valeurs NaN par la valeur la plus vue dans la colonne.

In [ ]:
df_train_final = df_train_final.apply(lambda x:x.fillna(x.value_counts().index[0]))
df_train_final.info()
df_train_final.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77982 entries, 0 to 77981
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   booking_id                    77982 non-null  float64
 1   people                        77982 non-null  float64
 2   purpose                       77982 non-null  object 
 3   gender                        77982 non-null  float64
 4   status                        77982 non-null  object 
 5   is_required_prepay_satisfied  77982 non-null  float64
 6   return90                      77982 non-null  float64
 7   lunch_year                    77982 non-null  float64
 8   lunch_month                   77982 non-null  float64
 9   lunch_day                     77982 non-null  float64
 10  lunch_hour                    77982 non-null  float64
 11  lunch_minute                  77982 non-null  float64
 12  reservation_date_year         77982 non-null  float64
 13  r

booking_id                      float64
people                          float64
purpose                          object
gender                          float64
status                           object
is_required_prepay_satisfied    float64
return90                        float64
lunch_year                      float64
lunch_month                     float64
lunch_day                       float64
lunch_hour                      float64
lunch_minute                    float64
reservation_date_year           float64
reservation_date_month          float64
reservation_date_day            float64
reservation_date_hour           float64
reservation_date_minute         float64
is_hotel                        float64
currency                         object
city                             object
cityarea                         object
opening_hours                    object
good_for_family                 float64
accept_credit_card              float64
parking                         float64


On passe booking_id en index.

In [ ]:
df_train_final=df_train_final.set_index('booking_id')

# Transformation des valeurs catégorielles

In [ ]:
train_dummies = None
train_dummies = pd.get_dummies(df_train_final, columns=['purpose', 'status','city', 'cityarea','residence','opening_hours'])
train_dummies.shape
#train_dummies.head()

(77982, 644)

# Création fonction

Fonction qui permet de recuperer l'ancienneté du membre sur le site de reservation.

In [ ]:
def anciennete_member(row):
  now=date(2020,5,3)
  date_row=date(row['mbr_record_date_year'],row['mbr_record_date_month'],row['mbr_record_date_day'])
  return (now-date_row).days


Fonction qui permet de recuperer l'ancienneté du restaurant sur le site de reservation.

In [ ]:
def anciennete_restau(row):
  now=date(2020,5,3)
  date_row=date(row['restau_add_date_year'],row['restau_add_date_month'],row['restau_add_date_day'])
  return (now-date_row).days


Fonction qui permet de recuperer l'ancienneté du membre sur le site de reservation au moment de la reservation.

In [ ]:
def anciennete_member_reserv(row):
  date_reservation=date(int(row['reservation_date_year']),int(row['reservation_date_month']),int(row['reservation_date_day']))
  date_member=date(int(row['mbr_record_date_year']),int(row['mbr_record_date_month']),int(row['mbr_record_date_day']))
  return (date_reservation-date_member).days


Fonction qui permet de recuperer l'ancienneté du membre sur le site de reservation au moment où il est venu manger.

In [ ]:
def anciennete_member_lunch(row):
  date_lunch=date(int(row['lunch_year']),int(row['lunch_month']),int(row['lunch_day']))
  date_member=date(int(row['mbr_record_date_year']),int(row['mbr_record_date_month']),int(row['mbr_record_date_day']))
  return (date_lunch-date_member).days

Fonction qui permet de recuperer la saison à laquelle le client est venu manger.

In [ ]:
def saison(row):
  if ((row['lunch_month']==3 and 20<row['lunch_day']) or row['lunch_month']==4 or row['lunch_month']==5 or (row['lunch_month']==6 and 20>=row['lunch_day'])):
    val='printemps'
  elif ((row['lunch_month']==6 and 20<row['lunch_day']) or row['lunch_month']==7 or row['lunch_month']==8 or (row['lunch_month']==9 and 20>=row['lunch_day'])):
    val = 'été'
  elif ((row['lunch_month']==9 and 20<row['lunch_day']) or row['lunch_month']==10 or row['lunch_month']==11 or (row['lunch_month']==12 and 20>=row['lunch_day'])):
    val='automne'
  else:
      val = 'hiver'
  return val

Fonction qui permet de savoir pour quel repas le client est venu manger.

In [ ]:
def repas(row):
  if (5<=row['lunch_hour']<=11):
    val='matin'
  elif (12<=row['lunch_hour']<=17):
    val = 'midi'
  else:
    val='soir'
  return val

Fonction qui permet le nettoyage de la table df

In [ ]:
def cleaning(df):
  #Remplacement valeur Nan
  df['purpose'].replace(np.nan,'Non renseigné',inplace=True)

  #Traitement de la date à laquelle les personnes sont venues manger
  df['datetime']=pd.to_datetime(df['datetime'])
  df['lunch_year']=df['datetime'].map(lambda x: x.strftime('%Y')).astype(int)
  df['lunch_month']=df['datetime'].map(lambda x: x.strftime('%m')).astype(int)
  df['lunch_day']=df['datetime'].map(lambda x: x.strftime('%d')).astype(int)
  df['lunch_hour']=df['datetime'].map(lambda x: x.strftime('%H')).astype(int)
  df['lunch_minute']=df['datetime'].map(lambda x: x.strftime('%M')).astype(int)
  df['saison']=df.apply(saison, axis=1)
  df['repas']=df.apply(repas,axis=1)
  #Traitement de la date de reservation
  df=df.rename(columns={'cdate':'reservation_date'})
  df['reservation_date']=pd.to_datetime(df['reservation_date'])
  df['reservation_date_year']=df['reservation_date'].map(lambda x: x.strftime('%Y')).astype(int)
  df['reservation_date_month']=df['reservation_date'].map(lambda x: x.strftime('%m')).astype(int)
  df['reservation_date_day']=df['reservation_date'].map(lambda x: x.strftime('%d')).astype(int)
  df['reservation_date_hour']=df['reservation_date'].map(lambda x: x.strftime('%H')).astype(int)
  df['reservation_date_minute']=df['reservation_date'].map(lambda x: x.strftime('%M')).astype(int)
  df=df.drop(labels=['reservation_date'],axis=1)

  #Conversion colonne gender en colonne binaire
  Gender = {'M': 1,'F': 0}
  df.gender = [Gender[item] for item in df.gender] 

  df_train_restaurant=pd.merge(df,df_restaurants,on=['restaurant_id'],how='outer')
  df_train_merge=pd.merge(df_train_restaurant,df_members,on=['member_id'],how='outer')
  df_train_final=df_train_merge[df_train_merge['booking_id'].notnull()]
  #Suppression des colonnes inutiles
  df_train_final=df_train_final.drop(labels=['member_id','restaurant_id','name','tel','abbr','gender_y','country','cityarea'],axis=1)
  df_train_final=df_train_final.drop(labels=['datetime'],axis=1)
  df_train_final=df_train_final.rename(columns={'gender_x':'gender'})

  #Remplacement valeur Nan
  df_train_final['residence']=df_train_final['residence'].fillna('0')
  df_train_final['price1']=df_train_final['price1'].fillna(df_train_final['price1'].mean())
  df_train_final['price2']=df_train_final['price2'].fillna(df_train_final['price2'].mean())
  df_train_final['age']=df_train_final['age'].fillna(df_train_final['age'].mean())
  df_train_final = df_train_final.apply(lambda x:x.fillna(x.value_counts().index[0]))
  df_train_final['anciennete_member_lunch']=df_train_final.apply(anciennete_member_lunch,axis=1)
  df_train_final['anciennete_member_reserv']=df_train_final.apply(anciennete_member_reserv,axis=1)


  #Booking_id en index
  df_train_final['booking_id']=df_train_final['booking_id'].astype(int)
  df_train_final=df_train_final.set_index('booking_id')

  return df_train_final





Fonction qui permet de transformer les colonnes de type string en colonne binaire grâce à get_dummies().



In [ ]:
def dummies(df):
  # Rassemblement de valeurs
  df = df.replace(dict.fromkeys(['Repas en famille','Repas en famille,盡量靠W','Repas en famille-生日','Family,Gathering',"Repas en f"], 'Famille'))
  df = df.replace(dict.fromkeys(['Dîner entre amis','Friends,Reunion'], 'Amis'))
  df = df.replace(dict.fromkeys(["Fête d'anniversaire",'Birthday,Celebration','Birthday',"Célébration d'anniversaire"], 'Anniversaire'))
  df = df.replace(dict.fromkeys(['Sweet day','Date importante'], 'Date'))
  df = df.replace(dict.fromkeys(["Dîner d'affaires"], 'Affaire'))
  # Remplacer  par 'Autres'
  df = df.replace(dict.fromkeys(['Veuillez sélectionner','其他','慶生' 'Please,Select',
                                    '&#65533;&#40115;&#65533;&#65533;&#26813;&#65533;', '家人&#65533;'], 'Autre'))
  df = df.replace(dict.fromkeys(['new','ok'], 'ok'))
  dummies=None
  dummies=pd.get_dummies(df, columns=['purpose', 'status','residence','city','opening_hours','currency','saison','repas']) #country 'cityarea'
  return dummies

Ici on regroupe Train et Test dans une seule dataframe pour qu'on soit sur qu'apres le traitement des tables, elles possèdent le même nombre de colonne. Après le traitement on redécoupe la dataframe en deux pour récuperer Train et Test du début.

In [ ]:
test=pd.read_csv("Test.csv")
train=pd.read_csv("Train.csv")
train['istest']=0
test['istest']=1
df_total=pd.concat([train, test])

#clean
df_total=cleaning(df_total)
#dummies
df_total=dummies(df_total)

train=df_total[df_total['istest']==0]
test=df_total[df_total['istest']==1]
train=train.drop(labels=['istest'],axis=1)
test=test.drop(labels=['return90','istest'],axis=1)


In [ ]:
# missing values
total_test = test.isnull().sum().sort_values(ascending=False)
# show
pd.DataFrame(data={'Missing': total_test})

# Classification

On souhaite dans notre cas prédire si un client reviendra ou pas dans le restaurant. Il s'agit donc d'une **classification binaire**. 
- Régression logistique
- Random forest
- XGBoost

On peut également utiliser la cross-validation pour déterminer les paramètres optimaux de chaque modèle.

## Logistique Regression

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
descriptive= train.loc[:, train.columns != 'return90']
prediction=train['return90']

In [1]:
# Separation of features variables and predict variable

X_train, X_test, y_train, y_test = train_test_split(descriptive,prediction, train_size=(2/3))

NameError: ignored

In [ ]:
lr = LogisticRegression(class_weight=None,dual=False,fit_intercept=True,intercept_scaling=1,l1_ratio=None,max_iter=100,multi_class='auto',n_jobs=None,penalty='l1',random_state=22,solver='liblinear',tol=0.0001,verbose=2,warm_start=False)
modele = lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)[:,1]
importance = modele.coef_[0]
feature_positive=[]
feature_negative=[]
# summarize feature importance
for i,v in enumerate(importance):
  if v>0:
    feature_importance.append([descriptive.columns[i],v])
  else:
    feature_inutile.append([descriptive.columns[i],v])
	#print('Feature: %0d, Score: %.5f' % (i,v))
feature_importance=pd.DataFrame(feature_importance,columns=['name','score_importance'])
feature_inutile=pd.DataFrame(feature_inutile,columns=['name','score_importance'])
#print(feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True))
feature_importance=feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True)
feature_inutile=feature_inutile.sort_values(by='score_importance',ascending=False).reset_index(drop=True)
feature_importance.to_csv("feature_positive_logistique.csv",sep=',')
feature_inutile.to_csv("feature_negative_logistique.csv",sep=',')
plt.bar([x for x in range(len(importance))], importance)
plt.show()
#Scores
score_train = lr.score(X_train, y_train)
score_test = lr.score(X_test, y_test)
print("Score sur la base de test :", score_test)
print("Score sur la base d'apprentissage :", score_train)
roc_auc_score(y_test, y_pred)

Prediction sur le fichier Test.

In [ ]:
y_pred=lr.predict_proba(test)[:,1]
y_pred

In [ ]:
y_pred=pd.DataFrame(y_pred,index=test.index,columns=['return90'])
#y_pred.reset_index(level=0, inplace=True)
#y_pred.return90.value_counts(dropna=False)
print(y_pred)
y_pred.to_csv('y_pred_logistique.csv', sep = ',')

# XGBOOST

In [ ]:
# Separation of features variables and predict variable

X_train, X_test, y_train, y_test = train_test_split(descriptive,prediction, train_size=(2/3))

Un traitement des colonnes a été necessaire pour faire passer la dataframe dans le modele XGBoost. 
Ici on a utilisé un XGBoost sans tuning. 

In [ ]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]
# model
boost = xgb.XGBClassifier(seed=42)

#train
boost.fit(X_train,y_train)
# get importance
importance = boost.feature_importances_
feature_importance=[]
# summarize feature importance
for i,v in enumerate(importance):
  if v>0:
    feature_importance.append([descriptive.columns[i],v])
  else:
    feature_inutile.append([descriptive.columns[i],v])
	#print('Feature: %0d, Score: %.5f' % (i,v))
feature_importance=pd.DataFrame(feature_importance,columns=['name','score_importance'])
feature_inutile=pd.DataFrame(feature_inutile,columns=['name','score_importance'])
print(feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True))
feature_importance=feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True)
feature_inutile=feature_inutile.sort_values(by='score_importance',ascending=True).reset_index(drop=True)
feature_importance.to_csv("feature_importance_xgboost1.csv",sep=',')
feature_inutile.to_csv("feature_inutile_xgboost1.csv",sep=',')
plt.bar([x for x in range(len(importance))], importance)
plt.show()
# test
y_pred = boost.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_pred))

Permet d'afficher le premier arbre généré par le modele. 

In [ ]:
xgb.to_graphviz(boost)

Prediction sur le fichier Test.

In [ ]:
test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test.columns.values]
y_pred2 = boost.predict_proba(test)[:,1]
y_pred2
#print(roc_auc_score(y_test, y_pred))

Enregistrement sous forme de dataframe pour passer à Kaggle.

In [ ]:
y_pred2=pd.DataFrame(y_pred2,index=test.index,columns=['return90'])
print(y_pred2)
y_pred2.to_csv('y_pred_xgboost1.csv', sep = ',')

Ici on a utilisé un XGBoost avec tuning mais en faisant du tatônnement c'est-à-dire que l'on testait plusieurs combinaisons de paramètre.

In [ ]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]
# model
model = xgb.XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.04,  
                      colsample_bytree = 1,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=300, 
                      reg_alpha = 0.3,
                      max_depth=5, 
                      gamma=1)
# train
model.fit(X_train, y_train)

# get importance

importance = model.feature_importances_
feature_importance=[]
# summarize feature importance
for i,v in enumerate(importance):
  if v>0:
    feature_importance.append([descriptive.columns[i],v])
	#print('Feature: %0d, Score: %.5f' % (i,v))
feature_importance=pd.DataFrame(feature_importance,columns=['name','score_importance'])
print(feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True))
plt.bar([x for x in range(len(importance))], importance)
plt.show()
# test

y_pred2=model.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_pred2))

Prediction sur le fichier Test.

In [ ]:
test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test.columns.values]
y_pred2 = boost.predict_proba(test)[:,1]
y_pred2

Enregistrement de la dtaframe pour soumettre à Kaggle. 

In [ ]:
y_pred2=pd.DataFrame(y_pred2,index=test.index,columns=['return90'])
print(y_pred2)
y_pred2.to_csv('y_pred_xgboost2.csv', sep = ',')

# XGBOOST with RandomizedSearchCV

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

params = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]}

folds = 4
param_comb = 3

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

clf = RandomizedSearchCV(xgb.XGBClassifier(learning_rate=0.02, n_estimators=300, objective='binary:logistic',
                  silent=True, nthread=1), param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )
clf.fit(X_train, y_train)
#print('\n Best estimator:')
print(clf.best_estimator_)
print(clf.best_params_)




Resultat du RandomizedSearchCV 

In [ ]:
model=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=5,
              learning_rate=0.02, max_delta_step=0, max_depth=4,
              min_child_weight=10, missing=None, n_estimators=300, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.6, verbosity=1)

Recuperation des variables importantes


In [ ]:
# train
model.fit(X_train, y_train)

# get importance
importance = model.feature_importances_
feature_importance=[]
feature_inutile=[]
# summarize feature importance
for i,v in enumerate(importance):
  if v>0:
    feature_importance.append([descriptive.columns[i],v])
  else:
    feature_inutile.append([descriptive.columns[i],v])
	#print('Feature: %0d, Score: %.5f' % (i,v))
feature_importance=pd.DataFrame(feature_importance,columns=['name','score_importance'])
feature_inutile=pd.DataFrame(feature_inutile,columns=['name','score_importance'])
print(feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True))
feature_importance=feature_importance.sort_values(by='score_importance',ascending=False).reset_index(drop=True)
feature_inutile=feature_inutile.sort_values(by='score_importance',ascending=True).reset_index(drop=True)
feature_importance.to_csv("feature_importance.csv",sep=',')
feature_inutile.to_csv("feature_inutile.csv",sep=',')
plt.bar([x for x in range(len(importance))], importance)
plt.show()

# test
y_pred2=model.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_pred2))

Enregistrement de la dataframe avec prediction du test, pour soumettre à Kaggle. 

In [ ]:
y_pred3=model.predict_proba(test)[:,1]
print(y_pred3)
y_pred3=pd.DataFrame(y_pred3,index=test.index,columns=['return90'])
y_pred3.to_csv('y_pred_xgboost3.csv', sep = ',')

# Random Forest

In [ ]:
# Separation of features variables and predict variable

X_train, X_test, y_train, y_test = train_test_split(descriptive, prediction, train_size=(2/3))

In [ ]:
# model
clf = RandomForestClassifier(max_depth=10,n_estimators=100,max_features=6)

# train
clf.fit(X_train, y_train)

# test
clf.predict(X_test)

y_pred3=clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_pred3)

0.6724836843794686

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

Recherche des meilleurs paramètres pour la prédiction avec GridSearchCv.

In [ ]:
forest = RandomForestClassifier(random_state=42)

params_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

# cross-validation strategy
cv = KFold(n_splits=5, random_state=42)

# hyper-parameter search
forest_cv = GridSearchCV(forest, params_grid, cv=cv, verbose=1, n_jobs=4)
forest_cv.fit(X_train, y_train)

# best score
score = np.sqrt(-forest_cv.score(X_test,y_test))

In [ ]:
print('Best hyper-params: ' + str(forest_cv.best_params_))

In [ ]:
# model
clf = RandomForestClassifier(criterion='gini', max_depth= 4, max_features='auto', n_estimators=200)

# train
clf.fit(X_train, y_train)
# test
y_pred3=clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_pred3)

0.6788747392774241

Affichage des variables explicatives importantes

In [ ]:
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(descriptive.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: status_canceled      Importance: 0.18
Variable: status_ok            Importance: 0.14
Variable: anciennete_member_lunch Importance: 0.1
Variable: anciennete_member_reserv Importance: 0.08
Variable: price2               Importance: 0.05
Variable: age                  Importance: 0.04
Variable: lunch_year           Importance: 0.03
Variable: reservation_date_year Importance: 0.03
Variable: is_hotel             Importance: 0.02
Variable: price1               Importance: 0.02
Variable: purpose_Autre        Importance: 0.02
Variable: people               Importance: 0.01
Variable: is_required_prepay_satisfied Importance: 0.01
Variable: lunch_month          Importance: 0.01
Variable: reservation_date_month Importance: 0.01
Variable: reservation_date_day Importance: 0.01
Variable: good_for_family      Importance: 0.01
Variable: wifi                 Importance: 0.01
Variable: restau_add_date_year Importance: 0.01
Variable: mbr_record_date_year Importance: 0.01
Variable: mbr_record_da